In [1]:
%load_ext autoreload
%autoreload 2

In [4]:
from plaid.datasets import FunctionOrganismDataModule
import torch
import webdataset as wds

ds = wds.WebDataset("/data/lux70/data/pfam/compressed/j1v1wv6w/val/shard{0000..0863}.tar")
# ds = wds.WebDataset("/data/lux70/data/pfam/compressed/j1v1wv6w/val/shard{0000..0002}.tar")

/homefs/home/lux70/micromamba/envs/flash_attn/lib/python3.11/site-packages/webdataset/compat.py:136: UserWarning: WebDataset(shardshuffle=...) is None; set explicitly to False or a number
  warnings.warn("WebDataset(shardshuffle=...) is None; set explicitly to False or a number")


In [ ]:
from plaid.datasets import make_sample, MetadataParser
from tqdm.notebook import tqdm

metadata_parser = MetadataParser()
max_length = 256

all_go_idxs = []
all_organism_idxs = []
all_local_paths = []
all_sample_ids = []
# all_embeddings = []
all_lengths = []

for datum in tqdm(ds):
    values = make_sample(datum, metadata_parser=metadata_parser, max_length=max_length)
    
    embedding, mask, go_idx, organism_idx, pfam_id, sample_id, local_path = values
    seq_lengths = mask.sum(axis=-1)
    
    # all_embeddings.append(embedding)
    all_lengths.append(seq_lengths)
    all_go_idxs.append(go_idx)
    all_organism_idxs.append(organism_idx)
    all_local_paths.append(local_path)
    all_sample_ids.append(sample_id)

# all_embeddings = np.stack(all_embeddings)

0it [00:00, ?it/s]

In [ ]:
# np.savez_compressed(f"/data/lux70/data/pfam/compressed/j1v1wv6w/val_stacked.npz", emb=all_embeddings)

In [ ]:
print(len(all_organism_idxs))
print(len(all_local_paths))
print(len(all_sample_ids))
print(len(all_go_idxs))

# print(all_embeddings.shape)

In [18]:
import pandas as pd

go_metadata_fpath = "/data/lux70/data/pfam/pfam2go.csv"
organism_metadata_fpath = "/data/lux70/data/pfam/organism_counts.csv"

go_df = pd.read_csv(go_metadata_fpath)
org_df = pd.read_csv(organism_metadata_fpath)

In [19]:
import pandas as pd

df = pd.DataFrame(
    {
        "GO_idx": all_go_idxs,
        "organism_index": all_organism_idxs,
        "local_paths": all_local_paths,
        "sample_ids": all_sample_ids
    }
)

In [20]:
df = df.merge(org_df, on="organism_index")

In [21]:
df = df.merge(go_df, on="GO_idx")

In [22]:
df = df.rename({"count": "GO_counts", "counts": "organism_counts"}, axis=1)

In [23]:
df.head()

,GO_idx,organism_index,local_paths,sample_ids,organism_id,organism_counts,pfam_id,GO_id,GO_term,GO_level,GO_counts
0,2130,472,/data/lux70/data/pfam/compressed/j1v1wv6w/val/...,sample001462,9ENTR,85933,PF15894,GO:0046325,negative regulation of glucose import,process,1
1,2130,215,/data/lux70/data/pfam/compressed/j1v1wv6w/val/...,sample001463,SERS3,4700,PF15894,GO:0046325,negative regulation of glucose import,process,1
2,2130,472,/data/lux70/data/pfam/compressed/j1v1wv6w/val/...,sample001464,9ENTR,85933,PF15894,GO:0046325,negative regulation of glucose import,process,1
3,2130,472,/data/lux70/data/pfam/compressed/j1v1wv6w/val/...,sample001465,9ENTR,85933,PF15894,GO:0046325,negative regulation of glucose import,process,1
4,2130,4,/data/lux70/data/pfam/compressed/j1v1wv6w/val/...,sample001466,9GAMM,2601336,PF15894,GO:0046325,negative regulation of glucose import,process,1
